# BCRP Dataset

The BCRP dataset represents the inhibition of the respective membrane transporters BCRP. 



reference:
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3596480/pdf/nihms431643.pdf

endpoint values are evidence of BCRP inhibition at 10uM

## Generate rdkit continuous descriptors, splitting dataset, and descriptor preprocessing

In [1]:
from rdkit import Chem
import pandas as pd
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
import os
currentDirectory = os.getcwd()
d = os.path.join(currentDirectory, "Datasets", "BCRP_0801.csv")
dataset = pd.read_csv(d, index_col = 0)
molecules = [Chem.MolFromSmiles(mol) for mol in dataset.SMILES]

calculator = MoleculeDescriptors.MolecularDescriptorCalculator([desc[0] for desc in Descriptors.descList])
X = pd.DataFrame([list(calculator.CalcDescriptors(mol)) for mol in molecules],
                     index=dataset.index,
                     columns=list(calculator.GetDescriptorNames()))

train_set_X, test_set_X = train_test_split(X, test_size=0.2, random_state=42)
train_set_y = dataset.loc[train_set_X.index]['endpoint'].values
test_set_y = dataset.loc[test_set_X.index]['endpoint'].values

In [3]:
from sklearn import pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
pipeline = pipeline.Pipeline([
        ('scaling', MinMaxScaler()),
        ('std_scaler', StandardScaler()),
    ])
train_X_prepared = pipeline.fit_transform(train_set_X)
test_X_prepared = pipeline.transform(test_set_X)

# Random Forest

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [5, 10, 20, 30, 40, 50],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'min_samples_split': [2],
    'n_estimators': [5, 100, 150, 200, 250, 300, 1000],
    'class_weight':['balanced']
}
# Create a based model
rfc = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rfc, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [7]:
# Fit the grid search to the data
grid_search.fit(train_X_prepared, train_set_y)
grid_search.best_params_

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:  1.2min finished
D:\anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'bootstrap': True,
 'class_weight': 'balanced',
 'max_depth': 50,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [12]:
from sklearn.metrics import confusion_matrix

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    tn, fp, fn, tp = confusion_matrix(test_labels, predictions).ravel()
    ppv = tp / (tp + fp)
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    ccr = (specificity + sensitivity) / 2
    
    print('Model Performance')
    print('ppv: {:0.2f}.'.format(ppv))
    print('specificity: {:0.2f}.'.format(specificity))
    print('sensitivity: {:0.2f}.'.format(sensitivity))
    print('ccr: {:0.2f}.'.format(ccr))
    
    return ccr

In [13]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_X_prepared, test_set_y)

Model Performance
ppv: 0.94.
specificity: 0.94.
sensitivity: 0.67.
ccr: 0.81.


In [14]:
best_grid = grid_search.best_estimator_
from sklearn.externals import joblib
joblib.dump(best_grid, "BCRP_rf_model_0806.pkl")
#my_model_loaded = joblib.load("my_model.pkl") 

D:\anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['BCRP_rf_model_0806.pkl']

# SVM 

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
# Create the parameter grid based on the results of random search 
param_grid = {
    'probability':[True],
    'class_weight':['balanced'],
    'kernel': ['rbf'],
    'gamma': [1e-2, 1e-3],
    'C': [1,10]}
# Create a based model
svm = SVC()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = svm, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [16]:
# Fit the grid search to the data
grid_search.fit(train_X_prepared, train_set_y)
grid_search.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:    1.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.1s finished
D:\anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 1,
 'class_weight': 'balanced',
 'gamma': 0.01,
 'kernel': 'rbf',
 'probability': True}

In [17]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_X_prepared, test_set_y)

Model Performance
ppv: 0.90.
specificity: 0.88.
sensitivity: 0.80.
ccr: 0.84.


In [18]:
best_grid = grid_search.best_estimator_
from sklearn.externals import joblib
joblib.dump(best_grid, "BCRP_svm_model_0806.pkl")

['BCRP_svm_model_0806.pkl']